In [1]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def find_sheet_name(sheet_names):
    cpi_sheet_names = list(filter(lambda x: "cpi" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    if len(sheet_names) == 43:
        return "30"

    if len(sheet_names) == 11:
        return "Sheet2"

    raise RuntimeError(sheet_names)

In [3]:
en_to_vn = {
    "CONSUMER PRICE INDEXES": "CHỈ SỐ GIÁ TIÊU DÙNG",
    "Food and catering services": "Hàng ăn và dịch vụ ăn uống",
    "Grain food": "Lương thực",
    "Foodstuff": "Thực phẩm",
    "Outdoor eating and drinking": "Ăn uống ngoài gia đình",
    "Drinks and tobacco": "Đồ uống và thuốc lá",
    "Textile, footgear and hats": "May mặc, mũ nón và giày dép ",
    "Housing and construction materials": "Nhà ở và vật liệu xây dựng",
    "Family appliances and tools": "Thiết bị và đồ dùng gia đình",
    "Medicaments and health service": "Thuốc và dịch vụ y tế",
    "Medical service": "Dịch vụ y tế",
    "Transport": "Giao thông",
    "Postal and communicational service": "Bưu chính viễn thông",
    "Education": "Giáo dục",
    "Educational service": "Dịch vụ giáo dục",
    "Culture, entertainment and tourism": "Văn hoá, giải trí và du lịch",
    "Others": "Hàng hóa và dịch vụ khác",
    "GOLD PRICE INDEXES": "CHỈ SỐ GIÁ VÀNG",
    "US DOLLAR PRICE INDEXES": "CHỈ SỐ GIÁ ĐÔ LA MỸ",
    "CORE INFLATION": "LẠM PHÁT CƠ BẢN",
}

In [5]:
def get_cpi(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Hàng ăn và dịch vụ ăn uống")
                | (df[df.columns[1]] == "Food and catering services")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Lương thực")
                | (df[df.columns[1]] == "Grain food")
                | (df[df.columns[1]] == " Lương thực")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if len(df.columns) == 7:
        df = df.drop(columns=[df.columns[1], df.columns[3], df.columns[4], df.columns[5]])
    elif len(df.columns) == 6:
        df = df.drop(columns=[df.columns[1], df.columns[3], df.columns[4]])
    elif len(df.columns) == 4 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[1], df.columns[3]])
        df["A"] = df[df.columns[1]]
    elif len(df.columns) == 5 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1], df.columns[2], df.columns[4]])
        df["A"] = df[df.columns[1]]
    else:
        raise RuntimeError(df.columns)

    df.columns = ["Hàng hóa", "M-YOY", "YTD-YOY"]

    df[df.columns[0]] = (
        df[df.columns[0]].replace(r"\s+", " ", regex=True).replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "CHỈ SỐ GIÁ TIÊU DÙNG")
                    | ((df[df.columns[0]] == "CONSUMER PRICE INDEXES"))
                ].index[0]
            )
        )
    )

    if len(df[df.columns[0]][df[df.columns[0]] == "CONSUMER PRICE INDEXES"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    df = df[pd.to_numeric(df["M-YOY"], errors="coerce").notnull()]
    df = df[pd.to_numeric(df["YTD-YOY"], errors="coerce").notnull()]

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    return df


xl = pd.ExcelFile("datas/2023-09-29-02-Bieu-so-lieu-9-thang-2023-1.xlsx")

sheet_name = find_sheet_name(xl.sheet_names)

get_cpi(xl, sheet_name, "2023-09-29")

,Hàng hóa,M-YOY,YTD-YOY,Date
10,CHỈ SỐ GIÁ TIÊU DÙNG,103.656606,103.158589,2023-09-29
12,Hàng ăn và dịch vụ ăn uống,102.873432,103.62232,2023-09-29
13,Lương thực,110.489819,104.852223,2023-09-29
14,Thực phẩm,101.165265,102.827401,2023-09-29
15,Ăn uống ngoài gia đình,104.004619,105.083794,2023-09-29
16,Đồ uống và thuốc lá,103.044436,103.484933,2023-09-29
17,"May mặc, mũ nón và giày dép",102.055235,102.304294,2023-09-29
18,Nhà ở và vật liệu xây dựng,107.331369,106.727963,2023-09-29
19,Thiết bị và đồ dùng gia đình,101.788104,102.27794,2023-09-29
20,Thuốc và dịch vụ y tế,100.578803,100.609915,2023-09-29


In [71]:
import os
import contextlib

dfs_cpi_origin = []
for file_name in sorted(
    os.listdir("./datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("./datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2014:
            continue

        xl = pd.ExcelFile(file_path)

        df_cpi = get_cpi(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_cpi_origin.append(df_cpi)
    except Exception as e:
        print(file_name)
        raise e

In [72]:
dfs_cpi = pd.concat(dfs_cpi_origin, ignore_index=True, axis=0)
dfs_cpi["Date"] = pd.to_datetime(dfs_cpi["Date"], dayfirst=False)
dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "M-YOY"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "M-YOY"].astype(float) - 100
)
dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "YTD-YOY"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "YTD-YOY"].astype(float) - 100
)
dfs_cpi

,Hàng hóa,M-YOY,YTD-YOY,Date
0,CHỈ SỐ GIÁ TIÊU DÙNG,3.656606,3.158589,2023-09-29
1,Hàng ăn và dịch vụ ăn uống,2.873432,3.62232,2023-09-29
2,Lương thực,10.489819,4.852223,2023-09-29
3,Thực phẩm,1.165265,2.827401,2023-09-29
4,Ăn uống ngoài gia đình,4.004619,5.083794,2023-09-29
...,...,...,...,...
2320,Dịch vụ giáo dục,12.45,12.45,2014-01-29
2321,"Văn hoá, giải trí và du lịch",2.9,2.9,2014-01-29
2322,Đồ dùng và dịch vụ khác,4.91,4.91,2014-01-29
2323,CHỈ SỐ GIÁ VÀNG,-24.43,-24.43,2014-01-29


In [73]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_cpi[
    (dfs_cpi["Hàng hóa"] == "CHỈ SỐ GIÁ TIÊU DÙNG")
    | (dfs_cpi["Hàng hóa"] == "Hàng ăn và dịch vụ ăn uống")
    | (dfs_cpi["Hàng hóa"] == "Nhà ở và vật liệu xây dựng")
    | (dfs_cpi["Hàng hóa"] == "Giao thông")
    | (dfs_cpi["Hàng hóa"] == "Giáo dục")
    | (dfs_cpi["Hàng hóa"] == "LẠM PHÁT CƠ BẢN")
]

fig = px.area(df_show, x="Date", y="M-YOY", color="Hàng hóa")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

/home/hien/.pyenv/versions/3.9.16/envs/vn-macroeconomics/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

